# Случайные леса на Титанике

Ахой, дамы и господа! Мы уже знакомились с игрушечными задачками для решения, когда данных было не так много и в них не было особо проблем: ни пропусков, ни строковых значений - все уже готово для модели. В этот раз мы возьмем данные посложнее и посмотрим, как работать с наиболее распространенными случаями предобработки!

В этот раз мы идем в плавание и планируем научиться предсказывать выживет ли человек или нет! В этом нам поможет набор данных с Kaggle - [Titanic: Machine Learning from Disaster](https://www.kaggle.com/c/titanic). Вы можете скачать данные прямо с сайта, ну а мы для примеров возьмем уже скачанный файл данных:

> 🚀 В этой практике нам понадобятся: `numpy==1.26.4, pandas==2.2.2, matplotlib==3.8.4, seaborn==0.13.2, scikit-learn==1.4.2` 

> 🚀 Установить вы их можете с помощью команды: `%pip install numpy==1.26.4 pandas==2.2.2 matplotlib==3.8.4 seaborn==0.13.2 scikit-learn==1.4.2` 


## Содержание

* [Базовый анализ данных](#Базовый-анализ-данных)
  * [Анализ признаков](#Анализ-признаков)
  * [Заполнение пропусков](#Заполнение-пропусков)
  * [Кодирование признаков](#Кодирование-признаков)
* [Полноценная предобработка](#Полноценная-предобработка)
* [Разработка модели случайного леса](#Разработка-модели-случайного-леса)
* [Важность признаков](#Важность-признаков)
* [Расширенный анализ данных](#Расширенный-анализ-данных)
  * [Больше графиков!](#Больше-графиков)
  * [Создание новых признаков](#Создание-новых-признаков)
  * [Группировка данных (binning)](#Группировка-данных-binning)
* [Поиск гиперпараметров](#Поиск-гиперпараметров)
* [Задачи](#Задачи)
* [Вопросы](#Вопросы)


In [ ]:
# Импорт необходимых модулей 
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Настройки для визуализации
# Если используется темная тема - лучше текст сделать белым
TEXT_COLOR = "black"

plt.rcParams["figure.figsize"] = (15, 10)
plt.rcParams["text.color"] = "black"
plt.rcParams["font.size"] = 14
plt.rcParams["axes.labelcolor"] = TEXT_COLOR
plt.rcParams["xtick.color"] = TEXT_COLOR
plt.rcParams["ytick.color"] = TEXT_COLOR

# Зафиксируем состояние случайных чисел
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [ ]:
# Перед этим скачайте его с kaggle - https://www.kaggle.com/c/titanic/data
# src_df = pd.read_csv(dataset_path)

На официальном сайте предоставлена информация по данным, сделаем краткое обобщение:
- `Survived` - (Целевая переменная) булевая переменная - выжил или нет;
- `PassengerId` - уникальный идентификатор пассажира; 
- `Pclass` - класс обслуживания;
- `Name` - имя пассажира;
- `Sex` - пол пассажира;
- `Age` - возраст пассажира (вещественное - возраст менее 1; х.5 - призблизительная оценка);
- `SibSp` - количество родственников на борту (братья, сестры, мужья, жены);
- `Parch` - количество родственников на борту (матери, отцы, дочери, сыновья);
- `Ticket` - номер билета;
- `Fare` - плата за проезд;
- `Cabin` - номер кабины;
- `Embarked` - порт посадки.

In [ ]:
print(f"Shape of data: {src_df.shape}")
print(src_df.columns)

In [ ]:
src_df.info()

Видим 891 запись, 12 колонок: 11 колонок данных и целевая переменная. Как видно из `.info()` - данные имеют пропуск и имеются колонки с типом `object`, что означает наличие строковых (а то и еще каких) данных. Давайте разберемся, как нам сделать базовую предобработку, чтобы построить первую модель!

## Базовый анализ данных

В этой части мы пройдемся по дополнительным методам подготовки данных для построения базовой модели (к тем, что мы изучили ранее). Целью базового анализа является первое знакомство с данными и подготовка к формату, который позволяет модели работать с данными - численная 2D матрица.

### Анализ признаков

При работе с данными первым делом необходимо понять, что значит каждый признак, какие в нем есть значения и пригодится ли он в работе. Для начала самыми подозрительными признаками являются признаки со строчными или целочисленными значениями, которые имеют слишком много уникальных значений. Как правило к категориальным такое отнести уже сложно, поэтому такие признаки чаще всего исключаются. Для примера возьмем признак `PassengerId`. С виду это какой-то индетификацтор, но взглянем на количество уникальных знчений:

In [ ]:
print(src_df["PassengerId"].nunique())
print(src_df["PassengerId"].count())
src_df["PassengerId"].head()

Как видно, абсолютно все значения уникальны, что означает невозможность применения данного признака в работе и он должен быть исключен. Аналогично имеет смысл проверить остальные признаки. Вещественные значения признаков часто также имеют очень много уникальных значений, но их либо сразу относят к типу признаков **непрерывные (численные)**, либо проводят более глубокий анализ.

Полезной практикой является проводить в начале унивариативный анализ, показывающий характеристики каждого признака и что с ним можно/нужно сделать в качестве первого этапа базового анализа.

Для колонок с типом `object` полезно воспользоваться методом `Series.describe()`:

In [ ]:
src_df["Name"].describe()

**Вывод:** самым первым этапом базового анализа является анализ признаков, который проводится по каждому признаку с целью определить применимость, необходимость коррекции и другие особенности и проблемы.

### Заполнение пропусков

Для просмотра количества пропусков удобно воспользоаться проверкой на `null` и затем вывести сумму по колонкам:

In [ ]:
src_df.isnull().sum()

По результатам отображения количества пропусков видно, что данные имеют пропуски и требуется произвести их обработку, так как данные в таком виде не могут быть применены для построения модели.

Существует огромное количество возможных вариантов работы с пропусками (https://scikit-learn.org/stable/modules/impute.html):
- Исключение признаков (колонок), имеющих пропуски;
- Исключение записей (строк), имеющих пропуски;
- Заполнение пропусков средним/медианным значением признака (**Унивариативное** заполнение - Используется единственный признак);
- Заполнение пропусков наиболее частым значением признака (мода);
- Заполнение путем построения регрессионной модели по остальным признакам (**Мультивариативное** заполнение - используется несколько признаков);
- и т.д.

Так, например, признак `Age` имеет 177 пропущенных значений, что является достаточно большим количеством, чтобы исключить записи с пропусками. Признак также невозможно исключить, так как он имеет информативный характер (возраст часто связывают со способностью к выживанию). Таким образом, можно воспользоваться `sklearn.impute.SimpleImputer` со стратегией заполнения `mean`, чтобы заполнить пропущенные данные на основе статистики остальных данных данного признака.

In [ ]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy="mean")
# Двойные скобки использованы, чтобы передать в fit() 2D массив
X_in = src_df[["Age"]]
print(X_in.shape)

src_df["Age"] = imp.fit_transform(X_in)
src_df.isnull().sum()

Вот так мы убедились в том, что метод работает! Исходя из распределения данных лучше выбирать стратегию по следующему признаку:
- Гауссово распределение - стратегия `mean`, так как в нормальном расрпделении наиболее частое ~ среднее значение;
- Ненормальное распределение - стратегия `meadian`, чтобы получить близкое к наиболее частому значению.

Для категориальных признаков наиболее простым методом является стратегия `most_frequent`, когда берется наиболее частое значение:

In [ ]:
imp = SimpleImputer(strategy="most_frequent")
src_df["Embarked"] = imp.fit_transform(src_df[["Embarked"]]).reshape(-1)

src_df.isnull().sum()

Другим методом работы с пропусками является исключение признаков из-за слишком большого количества пропусков:


In [ ]:
src_df["Cabin"].isnull().sum()/src_df.shape[0]

77% - это слишком большое количество пропусков, чтобы пытаться заполнить!

### Кодирование признаков

Как уже ранее обсуждалось, признаки бывают разные:
- Непрерывные (численные) - вещественные или целочисленные (чаще всего представляются типом `float` и `int`);
- Категориальные - могут быть представлены строками или числами с небольшим количеством уникальных значений, они могут быть разделены на следующие подтипы:
    - Номинальные - значения признаков ограничены группой возможных значений (красный/синий/зеленый);
    - Бинарные - те же номинальные, но всего две группы (Да/Нет, Правда/Ложь);
    - Последовательные - те же номинальные, но еще группы имеют порядок (плохой/хороший/отличный).

Непрерывные признаки с точки зрения базового анализа не требуют особой предобработки, так как они уже представлены числами. Можно лишь попробовать провести их стандартизацию.

С категориальными часто бывает чуть сложнее. Наибольшую проблему составляют признаки, которые представлены строками, например признак `Embarked`:

In [ ]:
src_df.info()
src_df["Embarked"].value_counts()

Этот признак представлен в данных типом `object` - строка, при этом мы видим, что уникальных значений мало, а это наводит на мысль, что признак - категориальный. Теперь важно понять, имеют ли значения порядок или они независимы? В данном случае логично предположить, что независимы, поэтому присваиваем тип - категориальные номинальный.

Но мало просто присвоить тип, нам же нужно подготовить данные для модели, а значит как-то представить в численном виде эти строки.

Для этого нам может помочь кодирование One-Hot! Давайте посмотрим, как это делается в `sklearn`:

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Мы отключим создание разреженного представления, но оно оптимальнее для хранения
# Поэтому для отладки лучше использовать и проверять dense представление,
#   а для работы в конечном представлении - sparse
oh_enc = OneHotEncoder(sparse_output=False)

# Любой энкодер ненавидит пропуски в данных, поэтому перед использованием
#   заполните пропуски в данных
X_sample = src_df[["Embarked"]]
print(X_sample.shape)

oh_enc.fit(X_sample)

In [ ]:
# Можно проверить, какие есть категории
oh_enc.categories_

In [ ]:
# Также посмотреть, что происходит после кодирования с данными
X_sample_ohe = oh_enc.transform(X_sample)

print(X_sample[:6])
print(X_sample_ohe[:6])

> Теперь попробуйте поменять флаг `sparse` на `True` и посмотреть на результат кодирования. *Dense* (плотное) представление матрицы - это то, к чему мы привыкли, но есть и более экономное - *sparse* (разреженное). В этом случае матрица представлена в виде списка пар (или `dict`), в котором первым элементом (или ключем) обозначается положение в матрице, а вторым - значение. В случае с OHE кодированием sparse представление - дело обычное!


Еще одним важным аргументом является поведение энкодера при поступлении новых данных, которые он раньше не видел. Тут многое зависит от задачи, но можно сделать так, чтобы он выдавал ошибку или игнорировал новую категорию, которая не была в обучающих данных:



In [ ]:
# Кидаем исключение, если появилась ранее невиданная категория
oh_enc = OneHotEncoder(sparse_output=False, handle_unknown="error")
oh_enc.fit(src_df[["Embarked"]])

# Воспользуемся try-except, чтобы поймать ошибку
try:
    print(oh_enc.transform(np.array([["K"]])))
except Exception as e:
    print(f"Error happened: {e}")

In [ ]:
# Или просто игнорируем
oh_enc = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
oh_enc.fit(src_df[["Embarked"]])

# Воспользуемся try-except, чтобы поймать ошибку
try:
    print(oh_enc.transform(np.array([["K"]])))
except Exception as e:
    print(f"Error happened: {e}")

1. Первый способ (ошибка) - удобен для проверки, что данные соответсвуют формату и все происходит так как надо - жесткая логика. 
2. Второй вариант (игнорировать) - более мягкая, но тут надо учитывать, что при обучении модель должна уметь обрабатывать ранее невиданные данные. Такой способ удобен, когда мы сами задаем список категорий:

In [ ]:
# Или просто игнорируем
oh_enc = OneHotEncoder(sparse_output=False, handle_unknown="ignore", categories=[["C", "Q"]])
X_sample = src_df[["Embarked"]]
oh_enc.fit(X_sample)

X_sample_ohe = oh_enc.transform(X_sample)

print(X_sample[:6])
print(X_sample_ohe[:6])

## Полноценная предобработка

Когда мы разобрались с тем, как нужно обработать признаки, мы можем поступить двумя способами:
- Написать свой код предобработки, протестировать, сохранить категории при кодировании, параметры стандартизации и другие этапы;
- Воспользоваться готовыми инструментами, которые делают все действия и на этапе `.fit()` вычисляют и запоминают параметры, чтобы далее во время `.transform()` их применять!

Первый способ подходит, когда нет готового инструмента, но `sklearn` имеет огромный арсенал по предобработке, а также можно посмотреть другие фреймворки!

Мы пойдем вторым способом и познакомимся с двумя полезными инструментами: [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) и [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html).

Пайплайн позволяет собирать несколько этапов обработки/моделей воедино, чтобы затем пользоваться им как единым целым! Для примера наша обработка категориальных признаков может состоять из двух этапов:

In [ ]:
from sklearn.pipeline import Pipeline

categorical_features = ["Sex", "Embarked"]

# Создаем обработчик категориальный признаков
# Так как имеются пропущенные данные в Embarked - 
#   создадим Pipeline для выполнения нескольких шагов
categorical_transformer = Pipeline(
    # Шаги в Pipeline указываются как кортежи, каждый из которых
    #   представляет собой (имя шага, трансформер)
    steps=[
        ("imp", SimpleImputer(strategy="most_frequent")),
        # Опять sparse для отладки = False
        ("enc", OneHotEncoder(sparse_output=False, handle_unknown="error")),                   
])

Далее, мы снова загрузим данные, чтобы восстановить те проблемы, которые решались до этого и воспользуемся реализацией пайплайна:

In [ ]:
df = pd.read_csv(dataset_path)

categorical_transformer.fit(df[categorical_features])

X_transformed = categorical_transformer.transform(df[categorical_features])
print(df[categorical_features][:3])
print(X_transformed[:3])

In [ ]:
# До конкретных шагов можно добраться через атрибут named_steps
categorical_transformer.named_steps["enc"].categories_

In [ ]:
# Или посмотреть названия признаков после кодирования
categorical_transformer.named_steps["enc"].get_feature_names_out(categorical_features)

> Если обратить внимание, то можно заметить, что `OneHotEncoder` генерирует признаки по количеству категорий, хотя признак `Sex` можно закодировать 0 или 1. Для этого есть аргумент в конструкторе `drop`, который управляет исключением лишних данных. Если его применить, то признак `Sex` в закодированном виде будет представлен всего одной колонкой, что выглядет логичнее.

Пайплайн очень удобен тем, что это полноценный объект настраиваемой предобработки, который можно один раз собрать и далее активно пользоваться! Более того, его можно сохранить в файл (сериализация) и затем переносить с сохраненными параметрами!

Также, пайплайн позволяет добраться до своих этапов, чтобы получить необходимые атрибуты!

Теперь перейдем к другому классу - преобразователь колонок! Его применение в том, чтобы также собирать шаги обработки, но уже указывая, на какие колонки, какая обработка. То есть в пайплайн мы явно передавали данные. `ColumnTransformer` позволяет настроить всю линию предобработки, указывая, какую колонку, чем обрабатывать:

In [ ]:
from sklearn.compose import ColumnTransformer

# Составляет список признаков для обработки
categorical_features = ["Sex", "Embarked"]
numeric_features = ["Age", "Fare"]

# Создаем обработчик категориальных признаков
categorical_transformer = Pipeline(
    steps=[
        ("imp", SimpleImputer(strategy="most_frequent")),
        ("enc", OneHotEncoder(handle_unknown="error")),                   
])

# Численные значения имеют пропуски, заполним стратегией медианы
numeric_transformer = SimpleImputer(strategy="median")

# Создаем конечный конвертер, который будет использован для 
#   предобработки
preprocessor = ColumnTransformer(
    # Список конвертеров, каждый кортеж содержит
    #   имя, конвертер и признаки, на которые он будет применен
    transformers=[
        ("cat", categorical_transformer, categorical_features),
        ("num", numeric_transformer, numeric_features)
    ],
    # Признаки, не указанные ни в одном из конвертеров будут удалены
    remainder="drop"
)

preprocessor.fit(df)

X_data = preprocessor.transform(df)
print(type(X_data))

На выходе конвертера получается числовая матрица numpy, которую можно уже передавать на вход модели. При этом трансформер колонок позволяет также получать доступ до своих составляющих через атрибут `named_transformers_`:

In [ ]:
# Получим объект пайплайна категориальных признаков
pipe = preprocessor.named_transformers_["cat"]
# Отобразим категории OHE для шага кодирования
print(pipe.named_steps["enc"].categories_)

При желании можно восстановить представление DataFrame, если из всех шагов собрать имена столбцов результирующей матрицы, но это нужно в редких случаях. А мы всё-таки сделаем для примера:



In [ ]:
# Для того, чтобы получить имена признаков, воспользуемся функцией
#   OneHotEncoder.get_feature_names()
# Чтобы ею воспользоваться, необходимо добраться до объекта через атрибуты
#   - ColumnTransformer.named_transformers_ + ключ имени
#   - Pipeline.named_steps + ключ шага
ohe_column_names = preprocessor \
    .named_transformers_["cat"] \
    .named_steps["enc"] \
    .get_feature_names_out(categorical_features)

recovered_feat_names = \
    list(ohe_column_names) + \
    list(numeric_features)

df_enc = pd.DataFrame(X_data, columns=recovered_feat_names)

df_enc.head()

In [ ]:
# Для сравнения выведем исходные данные
df[categorical_features + numeric_features].head()


Сутью данного инструмента является сбор инструментов обработки в единый объект уже после этапа поиска подходящих инструментов!

Аналогично, нынешний объект `preprocessor` можно разместить внутрь пайплайна вместе с моделью предсказания!

<p align="center"><img src="https://raw.githubusercontent.com/AleksDevEdu/ml_edu/master/assets/advanced/A4L-monitor.jpg"/></p>

## Разработка модели случайного леса

Подход с использование случайного леса (RandomForest) является одним из подходов группы под названием **ансамблирование**.

Ансамблирование - это парадигма, при которой большое количество слабых моделей собираются в группу и принятие конечного решения делается на основе голосования этих моделей. Основной девиз - много слабых моделей дают лучший результат, чем одна большая.

Подход RandomForest организует набор *решающих деревьев*. Решающее дерево - бинарное дерево, в котором узлами являются пороги одного из обученных признаков, листьями - предсказываемые классы.

Для примера обучим решающее дерево для задачи XOR.

In [ ]:
import numpy as np

# Пример решающего дерева на основе задачи XOR
X = np.array([
     [1, 1],
     [1, 0],
     [0, 1],
     [0, 0]
])

y = np.array([
     0,
     1,
     1,
     0
])

plt.figure(figsize=[5,5])
plt.scatter(X[y==1, 0], X[y==1, 1], marker="o")
plt.scatter(X[y==0, 0], X[y==0, 1], marker="x")
plt.xlabel("X[0]")
plt.ylabel("X[1]")
plt.xlim([-0.5, 1.5])
plt.grid()

from sklearn.tree import DecisionTreeClassifier, plot_tree

tree = DecisionTreeClassifier(random_state=42)
tree.fit(X, y)

plt.figure()
plot_tree(tree, filled=True, rounded=True, impurity=True, class_names=["0", "1"])


На рисунке представлена визуализация обученного дерева решения для нелинейной задачи XOR. Как видно, каждый узел определяет порог признака, так решение сходится до листьев, в котором и определяется конечное решение дерева (предсказанный класс).

Случайный лес работает по принципу набора таких деревьев:

<p align="center"><img src="https://raw.githubusercontent.com/AleksDevEdu/ml_edu/master/assets/advanced/A4L-forest.jpg"/></p>

Обучение деревьев происходит на основе алгоритма построения дерева (один из них - [CART](http://pages.stat.wisc.edu/~loh/treeprogs/guide/wires11.pdf)  ~ Classification and regression trees). Построение происходит по принципу поиска наилучших разделений пространства на основе одного из признаков для создания узла и дальнейшего роста.

Одной из важных особенностей случайного леса является то, что для обучения каждое дерево из леса получает не полную выборку, а лишь подвыборку из всей обучающей выборки. Такой принцип назван **Bagging**.

Деревья сами по себе очень чувствительны ко входным данным, так как могут бесконечно создавать узлы по признакам, чтобы максимально точно разделить классы. По факту, дерево очень легко переобучается на тех данных, которые даны для обучения (построения), если не ограничить глубину дерева.

Поэтому при построении каждого малого дерева используется подвыборка, куда могут попадать с повторениями случайные записи из данных со случайными признаками. Таким образом, каждая модель будет видеть лишь часть данных, при этом отдельно каждая слабая модель будет работать плохо, но в совокупности все модели будут давать более точную среднюю оценку в соответсвии с принятием решения по большинству голосов.

In [ ]:
from sklearn.model_selection import train_test_split

TRAIN_RATIO = 0.8

y_data = df["Survived"]

X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, 
    train_size=TRAIN_RATIO, 
    random_state=RANDOM_STATE,
    stratify=y_data
)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)

## Важность признаков

Обучение модели случайного леса позволяет получить оценку важности признаков! Для работы с показателями важности признаков достаточно воспользоваться атрибутом `RandomForestClassifier.feature_importances_`:

In [ ]:
def show_importance(model, feature_names, X):
    importances = model.feature_importances_
    for feat_imp, feat_name in zip(importances, feature_names):
        print(f"Feature: {feat_name} | {feat_imp}")

    indices = np.argsort(importances)[::-1]
    sorted_feat_names = [feature_names[ind] for ind in indices]

    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), importances[indices], color="b", align="center")
    plt.xticks(range(X.shape[1]), sorted_feat_names, rotation=70)
    plt.xlim([-1, X.shape[1]])
    plt.show()

show_importance(rf_clf, recovered_feat_names, X_train)

Такой график показывает, как признаки влияли на построение деревьев и насколько каждый из них важен по отношению к другим. 

> **ВАЖНО:** Несмотря на то, что цель графика - показать, насколько важны признаки для предсказания, **нельзя** полагаться лишь на результаты анализа лесом! Часто такая оценка важности смещена. Для более полного анализа в заданиях попробуйте воспользоваться подходом под названием *Feature Elimination*, который более точно позволяет оценить, какие признаки имеют высокое влияние на принимаемое решение модели! Суть подхода в том, что мы постепенно удаляем один за другим признаки из данных и оцениваем, как это повлияло на работу модели.

## Расширенный анализ данных

Результаты построения baseline модели как правило позволяют оценить результаты, которые можно получить наиболее быстрым способом. Дальнейших улучшений можно добиться как настраиванием модели и усложнением алгоритма, так и поиском "инсайтов" в данных, что позволит модели более просто понимать зависимости и принимать правильные решения. Для расширения знаний и подходов предлагаю ознакомиться с [хорошей статьей по EDA](https://towardsdatascience.com/predicting-the-survival-of-titanic-passengers-30870ccc7e8), а мы рассмотрим несколько основных способов проанализировать данные и сделать выводы.

### Больше графиков!

Один из самых простых способов проверить пользу признака - анализ нескольких признаков в совокупности, чтобы понять, какие взаимосвязи имеются и дает ли признак информацию. Начнем с того, что постараемся определить, кто чаще выживает - мужчины или женщины:

In [ ]:
sns.catplot(x="Sex", hue="Survived", kind="count", data=df)
plt.show()

Ответ на графике - выживаемость (Survived = 1) у женского пола больше, что означает необходимость использвания данного признака, так как он влияет на конечное решение! Если бы графики были ровные (все на одном уровне), мы бы не могли по полу человека сказать, кто скорее всего выживет, а значит такой признак бесполезен!

Еще одним интересный признак `Pclass`, класс обслуживания, можно проверить, влияет ли он на выживаемость:

In [ ]:
sns.barplot(x="Pclass", y="Survived", data=df)
plt.show()

Как видно, класс тоже вносит свое влияние в выживаемость: обслуживание первого класса более склонно к выживанию.

Также можно влияние признака проверить отображением через `sns.FacetGrid`, чтобы проверить все значения, связанные с несколькими признаками:

In [ ]:
FacetGrid = sns.FacetGrid(df, row="Embarked", height=4.5, aspect=1.6)
FacetGrid.map(sns.pointplot, "Pclass", "Survived", "Sex", palette=None,  order=None, hue_order=None)
FacetGrid.add_legend()
plt.show()

Как видно, в зависимости от порта посадки, а одних случаях (порты Q и S) женский пол имеет большие шансы на выживание, чего не скажешь о порте C. Тоже свой вклад в предсказания.

### Создание новых признаков

Помимо исключения признаков одним из широкоприменяемых подходов является создание новых признаков. Для примера, наличие двух признаков `SibSp` и `Parch` сообщает информацию о количестве родственников на борту. Таким образом, можно сформировать дополнительный признак `FamilySize`.

In [ ]:
# +1 - we are in family too
df["FamilySize"] = df["Parch"] + df["SibSp"] + 1

Новые признаки являются производными от исходных признаков, при этом зависимости не всегда могут являться линейными (например, из даты получить бинарную информацию, является ли день выходным или нет). Такие признаки могут помочь модели найти новые зависимостии и повысить точность.

> Как и ранее с генерацией полиномиальных признаков, можно проводить базовые операции над признаками (унивариативно или над несколькими). С одной стороны, можно делать операции на основе логических допущений - если вы понимаете, что новые признаки точно дадут новую информацию. В другой стороны, можно просто перебирать, пока не найдете такие операции, которые дадут прирост в точности, но это бывает долго и неэффективно.

Для примера после того, как мы создали свой признак "размер семьи", то можно отобразить единый график шансов выживания:

In [ ]:
sns.pointplot(x="FamilySize", y="Survived", data=df)
plt.show()

По такому графику можем видеть, что в зависимости от количества членов семьи меняются и шансы на выживание, при этом есть некоторая "критическая точка", что в четыре человека выжить больше шансов, нежели дальнейшее увеличение количества людей в семье на борту.

### Группировка данных (binning)

Одним из способов предобработки данных является группировка численные признаков для формирования категориального признака. Суть подхода в том, чтобы заменить непрерывное значение группами, каждая из которых описывает диапазон значений. Таким образом численное значение заменяется категориальным последовательным.

Такой подход является с одной стороны методом регуляризации, так как модель учится не на точных значениях, а на группах значений. С другой стороны, такой подход теряет часть информации.

Для примера разберем признак `Fare`:

In [ ]:
sns.displot(df["Fare"], bins=50, kde=True, height=7, aspect=1.5)
plt.show()

Раcпределение немного сдвинуто влево, также слева имеется небольшая мода. Для группировки воспользуется классом `KBinsDiscretizer`, который разделяет весь диапазон на заданное количество групп (бинов).

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

discr = KBinsDiscretizer(
    # Количество бинов    
    n_bins=10,
    # Способ кодирования - порядковый
    encode="ordinal",
)

# Двойные скобки для передачи DataFrame (2D данные)
df["Fare_groups"] = discr.fit_transform(df[["Fare"]])
# Отобразим границы бинов
print(discr.bin_edges_)

sns.catplot(x="Fare_groups", kind="count", data=df)
plt.show()

Группировка, как видно, привела сильно смещенное влево распределение к равномерному.

Более того, теперь мы можем визуально представить зависимость выживаемости от стоимости билета:

In [ ]:
sns.catplot(x="Fare_groups", y="Survived", data=df, kind="point")
plt.show()

или

In [ ]:
sns.catplot(x="Fare_groups", hue="Survived", data=df, kind="count")
plt.show()

Как видим, сгруппированный признак отображает определенную зависимость выживаемости и стоимости билета. Более высокая стоимость билета имеет больший шанс на выживание.

При этом обратите внимание, что сейчас деление на группы произведено по стратегии равного количества во всех группах. Можно попробовать сделать деление по равным диапазонам групп и сравнить!

## Поиск гиперпараметров

Как известно, многие модели имеют различные параметры, которые влияют на ход построения модели, максимальные и минимальные ограничения в структуре и др. При этом помним, что и сами модели имеют обучаемые параметры (для случая линейной регрессии, например, это веса при признаках). Те параметры, которые не участвуют в предсказании, но определяют архитектуру или другие внешние характеристики модели называют **гиперпараметрами**.

Уже не раз было необходимо произвести поиск и оценку этих гиперпараметров (в KNN - количество соседелей, в Ридж регрессии - $\alpha$). Ручной поиск хорошо справляется, когда имеется опыт и понимание работы модели, но также существуют и автоматизированные методы поиска - одним из них является **GridSearch**.

Основная суть метода в том, что для каждого гиперпараметра задается набор значений, которые требуется попробовать и далее создаётся набор из всех возможных комбинаций заданных гиперпараметров и их значений. Таким образом, исследуемые значения располагаются "как-бы на сетке".

В качестве реализации воспользуемся классом `sklearn.model_selection.GridSearchCV`, который реализует GridSearch с кросс-валидацией.

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {
    "max_depth": [1, 2, 4, 5, 7],
    "n_estimators": [1, 5, 10, 20, 40],
}

grid_search = GridSearchCV(
    # модель
    estimator=rf_clf,                   
    # сетка параметров
    #   может быть объектом dict 
    #   или list с несколькими dict внутри (несколько сеток)
    param_grid=parameters,              
    # кол-во фолдов для CV
    cv=5,                               
    # метрика для оценки - используем F1 
    scoring="f1_macro",   
)

grid_search.fit(X_train, y_train)

In [ ]:
# Для поиска параметров модели внутри пайплайна используется специальное именование:
#   <название шага>__<название параметра>
pipe = Pipeline(steps=[
    ("clf", RandomForestClassifier()),
])

# В названии два подчеркивания!
parameters = {
    "clf__max_depth": [1, 2, 4, 5, 7],
    "clf__n_estimators": [1, 5, 10, 20, 40],
}

grid_search = GridSearchCV(
    # пайплайн
    estimator=pipe,                   
    param_grid=parameters,              
    cv=5,                               
    scoring="f1_macro",   
)

grid_search.fit(X_train, y_train)

После того, как поиск закончен, можно посмотреть на сетку сгенерированных параметров через аттрибут `cv_results_` и ключ `params`.

In [ ]:
grid_search.cv_results_["params"]

Для получения наилучших параметров и оценки можно воспользоваться аттрибутами `best_params_` и `best_score_`.

In [ ]:
print(grid_search.best_params_)
print(grid_search.best_score_)

Аналогично для `ColumnTransformer`, для учета этапов необходимо использовать `__` для каждой агрегации операций (пайплайн или трансформер колонок). 

## Задачи

- Проведите базовый анализ данных, разделите данные на обучение/тест, разработайте baseline модель решающего дерева [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html), оцените работу модели, отобразите важности признаков;
- Разработайте модель линейной регрессии, оцените и сравните с моделью дерева;
- Произведите стандартизацию численных признаков и оцените (сравните) работу моделей с результатами обучения без стандартизации;
- Изучите влияние `max_depth` и `criterion` на показатели дерева, попробуйте 5 разных значений для каждого критерия, оцените с помощью кросс-валидации на обучающей выборке, сделайте таблицу;
- Проведите расширенный анализ данных, выберите наиболее приоритетные для классификации признаки, сравните выбранные признаки с показателями важности признаков, создайте новые признаки; В результате расширенного анализа обратите внимание на следующие особенностями:
    - Проведите создание новых признаков, добавляя каждый новый признак проведите оценку модели:
        - `FamilySize` - размер родственников на корабле;
        - `IsAlone` - является ли пассажир один на корабле или нет (бинарный признак);
        - `FarePerPerson` - оплата билета на человека в семье (воспользоваться `FamilySize`);
        - \*`NameTitle` (если придумаете как) - название титула, сформированое из признака `Name`, редкие титулы стоит объединить в одну группу;
    - Оцените работу модели при добавлении группировки признаков `Age`, `Fare`;
- Обучите модель дерева и модель леса [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) по подготовленным данным;
- Оцените влияние аргументов `max_depth` и `n_estimators` на точность модели (5 значений для каждого) с помощью кросс-валидации на обучающей выборке. Постройте таблицу зависимости метрик от величин;
- Определите наилучшие параметры для модели случайного леса через [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html);
- Определите наилучшие параметры для модели случайного леса через [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html);
- Постройте лучшие модели леса и решающего дерева и сравните их по показателям на выборке для теста.
- Примените подход [Recursive Feature Elimination](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html) на лучшую модель случайного леса. Сравните оценку важности признаков RFE и то, что показывает лес.
- Постройте ROC кривые моделей, сравните их и сделайте выводы:
    - лучшая модель на всех признаках;
    - лучшая модель только на топ-7 лучших признаках по RFE;
    - лучшая модель на топ-5 лучших признаках по RFE.

## Вопросы

- Какой метод оценки модели лучше использовать в данной работе? Разделение на обучение/тест или кросс-валидация? Можно/нужно ли применять их вместе?
- За что отвечают параметры `max_depth` и `n_estimators` в модели случайных лесов? Как они влияют на работу модели?
- В чем отличие GridSearch от RandomSearch?
- Как влияет стандартизация признаков на работу модели леса?
- Что такое "важность признака"? Есть ли аналоги показателя в моделях линейной и логистической регрессии?
